In [105]:
from mip import Model, xsum, maximize, BINARY
from gurobipy import *
import numpy as np

In [123]:
tests_per_week = 2
number_of_people = 20
f = np.zeros((7*number_of_people, 1))

# availability_matrix = # random availabilities
availability_matrix = np.ones((number_of_people, 7*number_of_people)) # pretend everyone is available

# Situation where each person is fully available
for i in range(len(availability_matrix)):
    if i > 0:
        for j in range(i*7):
            availability_matrix[i][j] = 0
    if i < len(availability_matrix):
        for j in range((i+1)*7, 7*number_of_people):
            availability_matrix[i][j] = 0
            
# Dealing with infeasibles... (not done yet)
# people_total_availability = np.sum(availability_matrix)

# Enforce that everyone gets two tests a week
Aeq = availability_matrix
beq = np.multiply(tests_per_week, np.ones((number_of_people,1)))

# This picks up the number of total tests (per?) person each day of the week
testing_capacity_matrix = np.zeros((7, 7*number_of_people))
for i in range(len(testing_capacity_matrix)):
    for j in range(i, 7*number_of_people, 7):
        testing_capacity_matrix[i][j] = 1
        
A = testing_capacity_matrix
# need a sanity check on value of b...should it be 6 for 20 people and 2 tests/wk?
b = np.ceil(np.multiply(number_of_people*tests_per_week/7, np.ones((7,1))))

# x is all zeros except for the 1 schedule choice per-person

# Ax <= b
# Aeq x = beq
# lb < x < ub
# x integers

x0 = np.ones((7*number_of_people, 1))
intcon = range(len(x0))
lb = np.zeros(len(x0))
ub = np.ones(len(x0))

# Gurobi optimization, still a little confused here.
m = Model()
m.setObjective(np.sum(x0), GRB.MAXIMIZE)

# c1 = m.addConstr(Aeq.dot(x0) == beq)
c2 = m.addConstr(A.dot(x0) <= b)

m.optimize()

TypeError: unsupported operand type(s) for -: 'bool' and 'NoneType'

In [129]:
Aeq.dot(x0) == beq

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])

In [126]:
type(beq)

numpy.ndarray

In [124]:
A.dot(x0)

array([[20.],
       [20.],
       [20.],
       [20.],
       [20.],
       [20.],
       [20.]])

In [118]:
beq.shape

(20, 1)

In [116]:
Aeq.dot(x0).shape

(20, 1)